In [3]:
import pandas as pd

# Ruta al archivo .dat
file_path = "files/GSS.dat"

# Definimos las posiciones (convertidas a índice base 0)
colspecs = [
    (0, 2), (2, 6), (6, 10), (10, 15), (15, 20),
    (20, 25), (25, 29), (29, 34), (34, 39), (39, 44),
    (44, 49), (49, 54), (54, 59), (59, 64), (64, 69),
    (69, 74), (74, 79), (79, 84), (84, 89), (89, 94),
    (94, 99), (99, 104), (104, 109), (109, 114), (114, 119),
    (119, 124)
]

column_names = [
    'BALLOT', 'WRKSTAT', 'INDUSTRY', 'AGE', 'SEX',
    'RINCOME', 'HAPPY', 'HAPMAR', 'HAPCOHAB', 'JOBLOSE',
    'SATJOB', 'FEJOBAFF', 'DISCAFFM', 'DISCAFFW', 'FEHIRE',
    'PROMTEFR', 'WKHARSEX', 'WKHAROTH', 'SATJOB1', 'SEXHAR',
    'HAPPY7', 'HAPUNHAP', 'UNHAPPY', 'HLTHDEP', 'ID_', 'YEAR'
]

# Cargar el archivo
gss = pd.read_fwf(file_path, colspecs=colspecs, names=column_names)

# Mostrar una muestra
gss.head()


,BALLOT,WRKSTAT,INDUSTRY,AGE,SEX,RINCOME,HAPPY,HAPMAR,HAPCOHAB,JOBLOSE,...,WKHARSEX,WKHAROTH,SATJOB1,SEXHAR,HAPPY7,HAPUNHAP,UNHAPPY,HLTHDEP,ID_,YEAR
0,1,3,-70,43,1,-90,2,-100,-100,4,...,2,2,2,-100,-100,2,-100,-70,1,2018
1,3,5,-70,74,2,-100,1,-100,-100,-100,...,-100,-100,-100,-100,-100,-100,1,-70,2,2018
2,2,1,-70,42,1,12,1,1,-100,4,...,2,2,2,-100,-100,1,-100,-70,3,2018
3,2,1,-70,63,2,12,1,1,-100,4,...,2,2,1,-100,-100,-100,2,-70,4,2018
4,3,5,-70,71,1,-100,2,-100,-100,-100,...,-100,-100,-100,-100,-100,-100,1,-70,5,2018




label values JOBLOSE   GSP009X;
label values SATJOB   GSP010X;
label values FEJOBAFF   GSP011X;
label values DISCAFFM   GSP012X;
label values DISCAFFW   GSP013X;
label values FEHIRE   GSP014X;
label values PROMTEFR   GSP015X;
label values WKHARSEX   GSP016X;
label values WKHAROTH   GSP017X;
label values SATJOB1   GSP018X;
label values SEXHAR   GSP019X;
label values HAPPY7   GSP020X;
label values HAPUNHAP   GSP021X;
label values BALLOT   GSP022X;
label values UNHAPPY   GSP023X;
label values HLTHDEP   GSP024X;

In [ ]:
value_labels = {
    "WRKSTAT": {
        1: "Working full time", 
        2: "Working part time", 
        3: "Job, but not at work",
        4: "Unemployed", 
        5: "Retired", 
        6: "In school", 
        7: "Keeping house", 
        8: "Other",
        -99: "Unk", 
        -98: "Unk", 
        -97: "Unk", 
        -100: "Unk"
    },
    "SEX": {
        1: "Male", 
        2: "Female",
        -99: "Unk", 
        -98: "Unk"
    },
    "HAPPY": {
        -96: 'Unk',
        -95: 'Unk',
        -90: 'Unk',
        -80: 'Unk',
        -70: 'Unk',
        -60: 'Unk',
        -50: 'Unk',
        -40: 'Unk',
        1: 'Very happy',
        2: 'Pretty happy',
        3: 'Not too happy',
        -100: 'Unk',
        -99: 'Unk',
        -98: 'Unk',
        -97: 'v'
    },

    "RINCOME": {
        12: "$25000 OR MORE",
        11: "$20000 - 24999",
        10: "$15000 - 19999",
        9: "$10000 - 14999",
        8: "$8000 TO 9999",
        7: "$7000 TO 7999",
        6: "$6000 TO 6999",
        5: "$5000 TO 5999",
        4: "$4000 TO 4999",
        3: "$3000 TO 3999",
        2: "$1000 TO 2999",
        1: "LT $1000",
        -99: "Unk",
        -98: "Unk",
        -90: "Unk",
        13: "Unk",
        -97: "Unk",
        -96: "Unk",
        -95: "Unk",
        -70: "Unk",
        -60: "Unk",
        -50: "Unk",
        -40: "Unk",
        98: "Unk",
        -100: "Unk",
        -80: "Unk"
    },

    "HAPMAR": {-96: 'Unk',
        -95: 'Unk',
        -90: 'Unk',
        -80: 'Unk',
        -70: 'Unk',
        -60: 'Unk',
        -50: 'Unk',
        -40: 'Unk',
        1: 'Very happy',
        2: 'Pretty happy',
        3: 'Not too happy',
        -100: 'Unk',
        -99: 'Unk',
        -98: 'Unk',
        -97: 'v'
    },

    "HAPCOHAB": {-96: 'Unk',
        -95: 'Unk',
        -90: 'Unk',
        -80: 'Unk',
        -70: 'Unk',
        -60: 'Unk',
        -50: 'Unk',
        -40: 'Unk',
        1: 'Very happy',
        2: 'Pretty happy',
        3: 'Not too happy',
        -100: 'Unk',
        -99: 'Unk',
        -98: 'Unk',
        -97: 'Unk'
    },
    "JOBLOSE": { 
        -96: 'Unk',
        -95: 'Unk',
        -90: 'Unk',
        -80: 'Unk',
        -70: 'Unk',
        -60: 'Unk',
        -50: 'Unk',
        -40: 'Unk',
        1: "Very likely",
        2: "Fairly likely",
        3: "Not too likely",
        4: "Not likely",
        5: "Leaving labor force",
        -100: 'Unk',
        -99: 'Unk',
        -98: 'Unk',
        -97: 'Unk'
    },
    
    "SATJOB": { 
        -96: 'Unk',
        -95: 'Unk',
        -90: 'Unk',
        -80: 'Unk',
        -70: 'Unk',
        -60: 'Unk',
        -50: 'Unk',
        -40: 'Unk',
        1: "Very satisfied",
        2: "Moderately satisfied",
        3: "A little dissatisfied",
        4: "Very dissatisfied",
        -100: 'Unk',
        -99: 'Unk',
        -98: 'Unk',
        -97: 'Unk'
    },
}


In [ ]:

# 4. Aplicar etiquetas al DataFrame
def apply_labels(df, label_dict):
    for column, labels in label_dict.items():
        if column in df.columns:
            df[column] = df[column].map(labels)
    return df



In [5]:
gss = apply_labels(gss, value_labels)
gss.head()

,BALLOT,WRKSTAT,INDUSTRY,AGE,SEX,RINCOME,HAPPY,HAPMAR,HAPCOHAB,JOBLOSE,...,WKHARSEX,WKHAROTH,SATJOB1,SEXHAR,HAPPY7,HAPUNHAP,UNHAPPY,HLTHDEP,ID_,YEAR
0,1,"Job, but not at work",-70,43,Male,Unk,Pretty happy,Unk,-100,4,...,2,2,2,-100,-100,2,-100,-70,1,2018
1,3,Retired,-70,74,Female,Unk,Very happy,Unk,-100,-100,...,-100,-100,-100,-100,-100,-100,1,-70,2,2018
2,2,Working full time,-70,42,Male,$25000 OR MORE,Very happy,Very happy,-100,4,...,2,2,2,-100,-100,1,-100,-70,3,2018
3,2,Working full time,-70,63,Female,$25000 OR MORE,Very happy,Very happy,-100,4,...,2,2,1,-100,-100,-100,2,-70,4,2018
4,3,Retired,-70,71,Male,Unk,Pretty happy,Unk,-100,-100,...,-100,-100,-100,-100,-100,-100,1,-70,5,2018


In [6]:
gss.columns

Index(['BALLOT', 'WRKSTAT', 'INDUSTRY', 'AGE', 'SEX', 'RINCOME', 'HAPPY',
       'HAPMAR', 'HAPCOHAB', 'JOBLOSE', 'SATJOB', 'FEJOBAFF', 'DISCAFFM',
       'DISCAFFW', 'FEHIRE', 'PROMTEFR', 'WKHARSEX', 'WKHAROTH', 'SATJOB1',
       'SEXHAR', 'HAPPY7', 'HAPUNHAP', 'UNHAPPY', 'HLTHDEP', 'ID_', 'YEAR'],
      dtype='object')

In [ ]:
gss = gss.rename(columns={
    'BALLOT': 'Boleta_entrevista',
    'WRKSTAT': 'Estado_laboral',
    'INDUSTRY': 'Industria_1970',
    'AGE': 'Edad',
    'SEX': 'Sexo',
    'RINCOME': 'Ingreso',
    'HAPPY': 'Felicidad_general',
    'HAPMAR': 'Felicidad_matrimonio',
    'HAPCOHAB': 'Felicidad_con_pareja',
    'JOBLOSE': 'Probabilidad_perder_trabajo',
    'SATJOB': 'Satisfacción_laboral',
    'FEJOBAFF': 'Opinión_contratación_preferencial_mujeres',
    'DISCAFFM': 'Discriminación_hombre_promoción',
    'DISCAFFW': 'Discriminación_mujer_promoción',
    'FEHIRE': 'Debería_contratar_promover_mujeres',
    'PROMTEFR': 'Promociones_son_justas',
    'WKHARSEX': 'Acoso_sexual_laboral_12m',
    'WKHAROTH': 'Amenazas_en_el_trabajo_12m',
    'SATJOB1': 'Satisfacción_general_trabajo',
    'SEXHAR': 'Ha_sufrido_acoso_sexual',
    'HAPPY7': 'Felicidad_autopercibida',
    'HAPUNHAP': 'Felicidad_actual',
    'UNHAPPY': 'Frecuencia_sentirse_desanimado_últimas_4_sem',
    'HLTHDEP': 'Se_sintió_deprimido_en_últimas_4_sem',
    'ID_': 'ID_entrevistado',
    'YEAR': 'Año'
})


In [ ]:
missing_codes = [-100, -99, -98, -97, -96, -95, -90, -80, -70, -60, -50, -40]
df.replace(missing_codes, np.nan, inplace=True)